<img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/2/2e/Logo_der_Technischen_Universit%C3%A4t_Berlin.svg/200px-Logo_der_Technischen_Universit%C3%A4t_Berlin.svg.png" decoding="async" width="150" height="112">

# Python for ecohydrology

Dr. Pedro Alencar (TU-Berlin Ökohydrologie & Landschaftsbewertung)

Class 2 - 01.12.2022


## Python - Crop Analysis

This notebook presents:

* WOFOST/PCSE

WOrld FOod STudies - WOFOST is a simulation model of growth and produciton of annual field crops. It is one of the key parts of the Global Yield Gap Atlas ([GYGA](http://www.yieldgap.org/)) and Monitoring Agricultural ResourceS ([MARS](https://joint-research-centre.ec.europa.eu/monitoring-agricultural-resources-mars_en)) initiatives.

It can be easily accessed with the Python Crop Simulation Environment ([PCSE](https://pcse.readthedocs.io/en/stable/#)).

For more information about WOFOST, visit their main [page](https://www.wur.nl/en/research-results/research-institutes/environmental-research/facilities-tools/software-models-and-databases/wofost.htm), where tips on [implementation](https://www.wur.nl/en/research-results/research-institutes/environmental-research/facilities-tools/software-models-and-databases/wofost/implementation-of-wofost.htm), and [downloads](https://www.wur.nl/en/research-results/research-institutes/environmental-research/facilities-tools/software-models-and-databases/wofost/downloads-wofost.htm), and [documentation](https://www.wur.nl/en/research-results/research-institutes/environmental-research/facilities-tools/software-models-and-databases/wofost/documentation-wofost.htm) are available.

To use WOFOST in Python, first it is necessary to install it:

In [ ]:
# pip install pcse

**Let's first import a few packages**

In [ ]:
import sys, os

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

import datetime

from IPython import display

In [ ]:
import pcse
print("This notebook was built with:")
print("python version: %s " % sys.version)
print("PCSE version: %s" %  pcse.__version__)

It is important to define a path to all input files. Make sure that the [folder](https://github.com/pedroalencar1/IntroToPython) `Data_WOFOST` is placed in a know location in your computer.

If you are using a computer with Windows, you should use two backslashes ("\\") to separate folders.

In [ ]:
data_dir = os.path.join("path/to/data/folder") # or "path\\to\\data\\folder"
# data_dir = os.path.join("")

In [ ]:
# for example (macOS):
data_dir = os.path.join("/Users/alencar/Library/CloudStorage/OneDrive-Personal/@_PostDoc/IntroToPython/Data_WOFOST")

In the data folder there are miltiple sub-folders with input data of WOFOST.

1. agro: has the agromanagement files that provide to the model the crop calendar and management activities (if any)
2. crop: has the crop parameter files for multiple calibrated crops, such as potato, wheat and corn
3. meteo: has the weather input files, containing geographic location and meteorological variables P, Tmax, Tmin, Irrad, WS.
4. soil: has the standard files with soil parameters


_Obs: sub-folders `site` and `output` will not be used now._

On soil parameters:
<img alt="" src="https://raw.githubusercontent.com/pedroalencar1/IntroToPython/master/Soil_parameters.png" decoding="async" width="850">

### 1. Read paramenters

#### **1.1. Crop parameters**

Here we can list all available crops in the WOFOST files

In [ ]:
os.path.join(data_dir,"crop")

In [ ]:
from pcse.fileinput import YAMLCropDataProvider
cropd = YAMLCropDataProvider(fpath = os.path.join(data_dir,"crop"))

cropd.print_crops_varieties()

# if you are using mac, you may need to delete the app file crops.yaml

In [ ]:
cropd = YAMLCropDataProvider(fpath = os.path.join(data_dir,"crop"))
# cropd.print_crops_varieties()

cropd.set_active_crop('maize', 'Grain_maize_201')
# print(yaml.dump(cropd, sort_keys=False, default_flow_style=False))

#### **1.2. Soil parameters**

The default values of soil parameters of WOFOST are derived from the European Soil Database V2.0 that separate the soils in 5 categoties, based on the containts of clay (C) and sand (S): 

1. Coarse (C < 18% and S > 65%)
2. Medium (18 < C < 35% and S > 15% or C < 18% and 15 < S < 65%)
3. Medium fine (C < 35% and S < 15%)
4. Fine (35 < C < 60%)
5. Very fine (C > 60%)

Additionally there is a class for Peat soils (no mineral texture, designated by the number *9* and not available in WOFOST)

For more, visit the [European soil satabase maps](https://esdac.jrc.ec.europa.eu/resource-type/european-soil-database-maps#)

**Note:** Soil class is selected by directly providing an input file (available at the [WOFOST GitHub page](https://github.com/ajwdewit/WOFOST/tree/master/soild)). The default values are loaded as a dictionary. The values then can be changed individually to locally calibrated parameters (if available) or for synthetic values (to assess uncertainty).

In [ ]:
from pcse.fileinput import CABOFileReader

soilfile = os.path.join(data_dir, 'soil', 'EC1.NEW')
soild = CABOFileReader(soilfile)

#### 1.3 Site parameters

Here we provide a few parameters relevant to the site location. Typically:
* WAV - Initial amount of water in rootzone *in excess of wilting point* in cm. Value is limited to SMLIM
* SMLIM - Maximum initial soil moisture in rooted zone (will be forced between SMW and SM0) (cm$^3$.cm$^{-3}$)
* CO2 - Atmospheric CO2 level in ppm. Default 360

In [ ]:
from pcse.util import WOFOST72SiteDataProvider
sited = WOFOST72SiteDataProvider(WAV=1) 
sited['CO2'] = 360
print(sited)

#### 1.4. Packaging all parameters

_"Finally, we need to pack the different sets of parameters into one variable using the `ParameterProvider`. This is needed because PCSE expects one variable that contains all parameter values."_

In [ ]:
from pcse.base import ParameterProvider
parameters = ParameterProvider(cropdata=cropd, soildata=soild, sitedata=sited)


**Note:** The parameters values are loaded as a dictionary. The values then can be easily changed here for fine tuning  and tests

To assess each one of the parameters dictionaries use:

* Soil: parameters._soildata
* Crop: parameters._cropdata
* Site: parameters._sitedata

In [ ]:
parameters._sitedata

Also at this stage you can easily change values of parameters

In [ ]:
parameters._sitedata['CO2'] = 396

parameters._sitedata

In [ ]:
parameters._soildata['RDMSOL'] = 120 #max root depth

### 2. Read agromanagement

The Agromanagement file provides to WOFOST basic information on the begining and end of the simulation for each year. 

In [ ]:
from pcse.fileinput import YAMLAgroManagementReader
agromanagement_file = os.path.join(data_dir, 'agro', 'basic.agro')
agromanagement = YAMLAgroManagementReader(agromanagement_file)
print(agromanagement)

The agromanagement file can be edited as a dictionary

In [ ]:
year = 2019

In [ ]:
agromanagement[0][datetime.date(year, 1, 1)] = agromanagement[0][datetime.date(9999, 1, 1)]
agromanagement[0].pop(datetime.date(9999, 1, 1))

print(agromanagement)

In [ ]:
agromanagement[0][datetime.date(year, 1, 1)]['CropCalendar']['crop_name'] = parameters._cropdata.current_crop_name
agromanagement[0][datetime.date(year, 1, 1)]['CropCalendar']['variety_name'] = parameters._cropdata.current_variety_name
agromanagement[0][datetime.date(year, 1, 1)]['CropCalendar']['crop_start_date'] = datetime.date(year, 4, 24)
agromanagement[0][datetime.date(year, 1, 1)]['CropCalendar']['crop_start_type'] = 'sowing' #or "emergence"
agromanagement[0][datetime.date(year, 1, 1)]['CropCalendar']['crop_end_date'] = datetime.date(year, 11, 26)
agromanagement[0][datetime.date(year, 1, 1)]['CropCalendar']['crop_end_type'] = "harvest" #or "maturity"
agromanagement[0][datetime.date(year, 1, 1)]['CropCalendar']['max_duration'] = 264

print(agromanagement)

### 3. Weather data

Currently the easiest way to load weather data into WOFOST is using Excel files. A script in `R` was prepared to help generating these files from DWD data, however, it is not possible to read them directly, as most cells are interpreted as text instead of numeric.
To solve this, the solution is to manually copy the generated files in R into a original (`basic_weather.xls`) with attention to make all cells as numeric (except dates).

In [ ]:
from pcse.fileinput import ExcelWeatherDataProvider

weatherfile = os.path.join(data_dir, 'meteo', 'potsdam1.xlsx') # potsdam1.xls
wdp = ExcelWeatherDataProvider(weatherfile)

print(wdp)

In [ ]:
df_inputs = pd.DataFrame(wdp.export())
df_inputs.columns

### 4. Processing WOFOST

Once the parameters and set, weather data provider loaded and agromanagement defined, the model can be run. 

Here we process the water limited and potential production scenarios

In [ ]:
from pcse.models import Wofost72_WLP_FD, Wofost72_PP
wofsim_wlp = Wofost72_WLP_FD(parameters, wdp, agromanagement) # water limited production
wofsim_pp = Wofost72_PP(parameters, wdp, agromanagement) # potential production

In [ ]:
wofsim_wlp.run_till_terminate()
df_results_wlp = pd.DataFrame(wofsim_wlp.get_output())
df_results_wlp = df_results_wlp.set_index("day")
df_results_wlp.tail()

In [ ]:
wofsim_pp.run_till_terminate()
df_results_pp = pd.DataFrame(wofsim_pp.get_output())
df_results_pp = df_results_pp.set_index("day")
df_results_pp.tail()

**Visualization**

In [ ]:
df_results_wlp['type'] = 'wlp'
df_results_pp['type'] = 'pp'

df = pd.concat([df_results_wlp, df_results_pp]).reset_index(drop=False)

df.tail()

In [ ]:
df_results_wlp['ET0'] = pd.DataFrame(wdp.export())['ET0']

In [ ]:
df['month'] = pd.DatetimeIndex(df['day']).month


In [ ]:
sns.set(rc={'figure.figsize':(15,15)})
from matplotlib.gridspec import GridSpec
from matplotlib.dates import DateFormatter

fig = plt.figure(constrained_layout=False)
gs = GridSpec(3, 2, figure=fig)
fig.subplots_adjust(hspace=0.5, wspace=0.3)
date_form = DateFormatter("%m")
# ax.xaxis.set_major_formatter(date_form)

ax1 = fig.add_subplot(gs[0, 0]).xaxis.set_major_formatter(date_form)
sns.lineplot(data=df, x = 'day', y = 'TAGP', hue = 'type', ci=None,
            palette = ['red', 'blue']).set(title='TAPG Gap {}'.format(year), xlabel = None, xticklabels=[])

ax2 = fig.add_subplot(gs[0, 1]).xaxis.set_major_formatter(date_form)
sns.lineplot(data=df, x = 'day', y = 'LAI', hue = 'type', ci=None,
            palette = ['red', 'blue']).set(title='LAI Gap {}'.format(year), xlabel = None, xticklabels=[])

ax3 = fig.add_subplot(gs[1, 0]).xaxis.set_major_formatter(date_form)
sns.lineplot(data=df, x = 'day', y = 'TWSO', hue = 'type', ci=None,
            palette = ['red', 'blue']).set(title='TWSO Gap {}'.format(year),xlabel = 'Month')

ax4 = fig.add_subplot(gs[1, 1]).xaxis.set_major_formatter(date_form)
sns.lineplot(data=df, x = 'day', y = 'TRA', hue = 'type', ci=None,
            palette = ['red', 'blue']).set(title='TRA Gap {}'.format(year),xlabel = 'Month')

ax5 = fig.add_subplot(gs[2, :]).xaxis.set_major_formatter(date_form)
sns.lineplot(data=df, x = 'day', y = 'SM', hue = 'type', ci=None,
            palette = ['red', 'blue']).set(title='SW Gap {}'.format(year),xlabel = 'Month')

plt.show()

In [ ]:
df_input = pd.DataFrame(wdp.export())
df_input = df_input[pd.DatetimeIndex(df_input['DAY']).year  == 2019]
df_input_plot = df_input[["DAY", "ET0", "RAIN", "TMAX", "TMIN"]]
df_input_plot["SM"] = df_results_wlp["SM"]
df_input_plot["TEMP"] = (df_input_plot["TMAX"] + df_input_plot["TMIN"])/2
df_input_plot["RAIN_mm"] =  df_input_plot["RAIN"]*10
df_input_plot["ET0_mm"] =  df_input_plot["ET0"]*10
df_input_plot.tail()

In [ ]:
sns.set(rc={'figure.figsize':(15,15)})
from matplotlib.gridspec import GridSpec
from matplotlib.dates import DateFormatter

fig = plt.figure(constrained_layout=False)
gs = GridSpec(3, 1, figure=fig)
fig.subplots_adjust(hspace=0.2, wspace=0.3)
date_form = DateFormatter("%m")

ax1 = fig.add_subplot(gs[0, 0]).xaxis.set_major_formatter(date_form)
sns.lineplot(data=df_input_plot, x = 'DAY', y = 'RAIN_mm', ci=None, color = 'blue'
            ).set(title='Precipitation', xlabel = None, xticklabels=[], ylabel = 'P (mm/day)')

ax2 = fig.add_subplot(gs[1, 0]).xaxis.set_major_formatter(date_form)
sns.lineplot(data=df_input_plot, x = 'DAY', y = 'TEMP', ci=None, color = 'blue'
            ).set(title='Temperature', xlabel = None, xticklabels=[])
sns.lineplot(data=df_input_plot, x = 'DAY', y = 'TMAX', ci=None, color = 'grey', linewidth = 0.6,
            ).set(title='Temperature', xlabel = None, xticklabels=[])
sns.lineplot(data=df_input_plot, x = 'DAY', y = 'TMIN', ci=None, color = 'grey', linewidth = 0.6,
            ).set(title='Temperature', xlabel = None, xticklabels=[], ylabel = 'T (Celcius)')



ax3 = fig.add_subplot(gs[2, 0]).xaxis.set_major_formatter(date_form)
sns.lineplot(data=df_input_plot, x = 'DAY', y = 'ET0_mm', ci=None, color = 'blue'
            ).set(title='Potential evapotranspiration',xlabel = 'Month',ylabel = 'ET0 (mm/day)')



plt.show()

In [ ]:
import plotly.graph_objects as go

# Create random data with numpy
import numpy as np
np.random.seed(1)

N = 100
random_x = np.linspace(0, 1, N)
random_y0 = np.random.randn(N) + 5
random_y1 = np.random.randn(N)
random_y2 = np.random.randn(N) - 5

# Create traces
fig = go.Figure()
fig.add_trace(go.bar(x=df_test['DAY'], y=df_test['RAIN'],
                    name='Rain'))
fig.add_trace(go.Scatter(x=df_test['DAY'], y=df_test['ET0'],
                    mode='lines',
                    name='ET0'))
fig.add_trace(go.Scatter(x=df_test['DAY'], y=df_test['TMAX'],
                    mode='lines', name='TMAX'))

fig.show()


In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=df_input_plot["DAY"], y=df_input_plot["TEMP"], name = 'Temp'),
    secondary_y=False,    
)

fig.add_trace(
    go.Bar(x=df_input_plot["DAY"], y=df_input_plot["RAIN_mm"], name="Rain"),
    secondary_y=True,
)
    

fig.update_layout(
    title_text="Double Y Axis Example"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="Temperature (Celcius)", secondary_y=False, range = [-20, 40])
fig.update_yaxes(title_text="Precipitation (mm)", secondary_y=True, range = [120, 0])



# fig = px.line(x=df_input_plot["DAY"], y=df_input_plot["TEMP"], color=px.Constant("Temperature"),
#               labels=dict(x="Date", y="Temperature (Celcius)", color="Legend"))
# fig.add_bar(x=df_input_plot["DAY"], y=df_input_plot["RAIN_mm"], name="Rain")
# fig.show()

# Exercise

For the same parameters, repeat the process for the years 2017 and 1982. 

* What happened in those years?
* Try changing other parameters and see the impact. Are there crops that suffer less with drought? What if you start the crop cycle earlier?

